# **Step 1**

### **(1) Using Heston and  Monte-Carlo Methods for option pricing**


In the Heston model, the diffusion of the stock price $ S_t $  is given by the following stochastic differential equations:

$$
\begin{equation*}
    S_t = S_{t-1} e^{\left( r - \frac{\nu_t}{2} \right) dt + \sigma \sqrt{\nu_t} dZ_1}
\end{equation*}
$$
and
$$
\begin{equation*}
    \nu_t = \nu_{t-1} + \kappa \left( \theta - \nu_{t-1} \right) dt + \sigma \sqrt{\nu_{t-1}}dZ_2
\end{equation*}
$$

Where:

- $ S_t $ is the stock or asset price at time $ t $.
- $ \nu_t $ the asset return variance at time $ t $.
- $ r $ is the risk-free interest rate.
- $ \kappa $ is the rate at which $ \nu_t $ reverts to its long-term mean $ \theta $.
- $ \sigma $ is a volatility parameter.
- $ dZ_1 $ and $ dZ_2 $ are regarded as two Brownian motion processes.

Monte Carlo methods multiple paths for the asset price and its variance for simulation. The concluding option price is the average of these simulated paths.

**Delta and Gamma**:

1. **Delta**: It is the rate of change of the option price with respect to the underlying asset's price. $$ \Delta \approx \frac{C(S_0 + \Delta S) - C(S_0)}{\Delta S} $$ where $ C $ is the option price and $ \Delta S $ is a small change in the stock price.

2. **Gamma**: It i the rate of change of Delta with respect to the underlying asset's price.$$ \Gamma \approx \frac{C(S_0 + \Delta S) - 2C(S_0) + C(S_0 - \Delta S)}{\Delta S^2} $$

The Heston model, above is a good candidate for modelling real world option prices

**Question 5**

Pricing ATM European call and put with Heston Model, using a correlation
value of -0.30.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations
dt = T / M  # Length of time step


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)
# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

def heston_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])  # Calculate the payoff for a put option

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

print("European Call Price under Heston using a correlation value of -0.30: ",np.round( heston_call_mc(S, 80, 0.055, 3/12, 0),2))
print("European Put Price under Heston using a correlation value of -0.30: ", np.round(heston_put_mc(S, 80, 0.055, 3/12, 0),2))


epsilon = 0.01
S1 = S0 + epsilon
S2 = S0 - epsilon

# Calculate deltas for perturbed asset prices using the Heston model
epsilon = 0.01
S1 = S0 + epsilon
S2 = S0 - epsilon
S3 = S0 + epsilon + epsilon + epsilon
S_perturbed_1 = Heston_paths(S1, r, V, 0, cho_matrix)
S_perturbed_2 = Heston_paths(S2, r, V, 0, cho_matrix)
S_perturbed_3 = Heston_paths(S3, r, V, 0, cho_matrix)

delta_call1 = (heston_call_mc(S_perturbed_1, 80, r, T, 0) - heston_call_mc(S_perturbed_2, 80, r, T, 0)) / (2 * epsilon)
delta_put1 = (heston_put_mc(S_perturbed_1, 80, r, T, 0) - heston_put_mc(S_perturbed_2, 80, r, T, 0)) / (2 * epsilon)

S_f = 81
Sf1 = S_f + epsilon
Sf2 = S_f - epsilon

delta_call1x = (heston_call_mc(S_perturbed_3, Sf1, r, T, 0) - heston_call_mc(S_perturbed_1, Sf2, r, T, 0)) / (2 * epsilon)
delta_put1x = (heston_put_mc(S_perturbed_3, Sf1, r, T, 0) - heston_put_mc(S_perturbed_1, Sf2, r, T, 0)) / (2 * epsilon)


# Calculate gamma using the Heston model
gamma_call1 = ((delta_call1 - delta_call1x) / (S_f - S0)).round(3)
gamma_put1 = ((delta_put1x - delta_put1) / (S_f - S0)).round(3)



European Call Price under Heston using a correlation value of -0.30:  2.85
European Put Price under Heston using a correlation value of -0.30:  2.84


We have used Monte Carlo simulation to price options under the Heston model. It includes functions for:
SDE_vol: Generates volatility paths via the Heston model's stochastic differential equation using Cholesky decomposition for correlated random numbers.
Heston_paths: Generates asset price paths by simulating the Heston model with given parameters.
Random_number_gen: Generates standard normal random numbers for simulations.
The main simulation:
Sets up Heston model parameters.
Simulates volatility and asset price paths.
Calculates European call and put option prices using Monte Carlo simulation.


Question 6

Pricing ATM European call and put with Heston Model, using a correlation
value of -0.70.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.7

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations
dt = T / M  # Length of time step


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)
# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

def heston_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])  # Calculate the payoff for a put option

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

print("European Call Price under Heston using a correlation value of -0.70: ",np.round( heston_call_mc(S, 80, 0.055, 3/12, 0),2))
print("European Put Price under Heston using a correlation value of -0.70: ", np.round(heston_put_mc(S, 80, 0.055, 3/12, 0),2))

# Calculate deltas for perturbed asset prices using the Heston model
epsilon = 0.01
S1 = S0 + epsilon
S2 = S0 - epsilon
S3 = S0 + epsilon*3
S_perturbed_1 = Heston_paths(S1, r, V, 0, cho_matrix)
S_perturbed_2 = Heston_paths(S2, r, V, 0, cho_matrix)
S_perturbed_3 = Heston_paths(S3, r, V, 0, cho_matrix)

delta_call2 = (heston_call_mc(S_perturbed_1, 80, r, T, 0) - heston_call_mc(S_perturbed_2, 80, r, T, 0)) / (2 * epsilon)
delta_put2 = (heston_put_mc(S_perturbed_1, 80, r, T, 0) - heston_put_mc(S_perturbed_2, 80, r, T, 0)) / (2 * epsilon)

S_f = 81
Sf1 = S_f + epsilon
Sf2 = S_f - epsilon

delta_call2x = (heston_call_mc(S_perturbed_3, Sf1, r, T, 0) - heston_call_mc(S_perturbed_1, Sf2, r, T, 0)) / (2 * epsilon)
delta_put2x = (heston_put_mc(S_perturbed_3, Sf1, r, T, 0) - heston_put_mc(S_perturbed_1, Sf2, r, T, 0)) / (2 * epsilon)



# Calculate gamma using the Heston model
gamma_call2 = ((delta_call2 - delta_call2x) / (S_f - S0)).round(3)
gamma_put2 = ((delta_put2x - delta_put2) / (S_f - S0)).round(3)


European Call Price under Heston using a correlation value of -0.70:  2.08
European Put Price under Heston using a correlation value of -0.70:  3.46


We have employed the Heston model with Monte Carlo simulations to price European call and put options for a given set of parameters. It first generates paths for both the volatility and underlying asset price processes. The calculated European call and put option prices under the Heston model showcase the valuation at a correlation of -0.7. Furthermore, the code approximates delta and gamma values by perturbing the asset price and observing the resulting changes in option prices, illustrating the sensitivity and curvature of the option prices concerning the underlying asset's movement.

Question 7




**7.a(Delta and gamma for Q5)**

In [ ]:


print("European Call Delta under Heston using a correlation value of -0.30: ",delta_call1.round(2))
print("European Put Delta under Heston using a correlation value of -0.30: ",delta_put1.round(2) )


print("European Call Gamma under Heston using a correlation value of -0.30: ",gamma_call1.round(3))
print("European Put Gamma under Heston using a correlation value of -0.30: ",gamma_put1.round(3) )


European Call Delta under Heston using a correlation value of -0.30:  0.54
European Put Delta under Heston using a correlation value of -0.30:  -0.45
European Call Gamma under Heston using a correlation value of -0.30:  0.503
European Put Gamma under Heston using a correlation value of -0.30:  0.483


**7.b(Delta and gamma for Q6)**

In [ ]:


print("European Call Delta under Heston using a correlation value of -0.70: ",delta_call2.round(2))
print("European Put Delta under Heston using a correlation value of -0.70: ",delta_put2.round(2) )


print("European Call Gamma under Heston using a correlation value of -0.70: ",gamma_call2.round(3))
print("European Put Gamma under Heston using a correlation value of -0.70: ",gamma_put2.round(3) )


European Call Delta under Heston using a correlation value of -0.70:  0.48
European Put Delta under Heston using a correlation value of -0.70:  -0.49
European Call Gamma under Heston using a correlation value of -0.70:  0.457
European Put Gamma under Heston using a correlation value of -0.70:  0.53


Explanation of Delta and Gamma:
Delta measures the sensitivity of an option's price to changes in the underlying asset's price. A positive delta indicates a call option, while a negative delta indicates a put option. Gamma, on the other hand, quantifies the rate of change of delta concerning changes in the underlying asset's price.
For both correlation scenarios (-0.30 and -0.70):
Call options have positive deltas, indicating a positive relationship with the underlying asset.
Put options have negative deltas, reflecting an inverse relationship with the underlying asset.
Gamma measures the change in delta concerning changes in the underlying asset's price. Both call and put options have positive gamma values. Gamma is highest near the at-the-money region and diminishes as options move deeper in or out of the money.


Question 8

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [ ]:
np.random.seed(42)

In [ ]:
S0 = 80
r = 0.055
sigma = 0.35
T = 3/12
Ite = 10000000  # Number of simulations (paths)
M = 1  # Number of steps
dt = T / M  # Time-step

lamb = 0.75  # Lambda of the model
mu = -0.6  # Mu
delta = 0.22  # Delta

In [ ]:
SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

In [ ]:
for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [ ]:
def merton_mc(S, K, r, T, t, type="C"):
    if (type=="C"):
      payoff = np.maximum(0, S[-1, :] - K)
    else:
      payoff = np.maximum(K - S[-1, :], 0)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
print("European Call Price under Merton: ", merton_mc(SM, 80, r, T, 0, type="C").round(2))

European Call Price under Merton:  8.92


In [ ]:
print("European Put Price under Merton: ", merton_mc(SM, 80, r, T, 0, type="P").round(2))

European Put Price under Merton:  7.38


I use a 10,000,000 (10 million) simulation on pricing option under Merton model since the pricing process does not involve any for loop, therefore the calculation process would be faster and could iterate more.

Process: First create a matrix of price SM, with (M+1) rows time steps, and (Ite) columns for each path. Then calculate r_j as the formula, and generate random z1, z2, and y. After that, create a stock price path loop over each time step to simulate the stock price and to make sure the stock price does not go below zero. Finally, the payoff is calculated based on the final stock prices and option types in the simulation paths. The function returns the present value of the average payoff as the option price.


Question 9

In [ ]:
lamb = 0.25  # Lambda of the model

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [ ]:
print("European Call Price under Merton with lambda 0.25: ", merton_mc(SM, 80, r, T, 0, type="C").round(2))

European Call Price under Merton with lambda 0.25:  7.01


In [ ]:
print("European Put Price under Merton with lambda 0.25: ", merton_mc(SM, 80, r, T, 0, type="P").round(2))

European Put Price under Merton with lambda 0.25:  5.84


When decreased 𝜆 (lambda), the option price also decreases. 𝜆 represents the Intensity (frequency) of the jump (shock) in stock prices. Higher 𝜆 means a higher chance that the stock price experience a negative jump (return). Therefore, when decreasing 𝜆, the stock experiences less jump, making it less volatile and not moving far, making the chance of being at or in the money at the expiration lower, hence the price is lower

Question 10

In [ ]:
epsilon = 0.01

In [ ]:
S1 = S0 + epsilon
S2 = S0 - epsilon

In [ ]:
SM1 = np.zeros((M + 1, Ite))
SM1[0] = S1
for t in range(1, M + 1):
    SM1[t] = SM1[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM1[t] = np.maximum(
        SM1[t], 0.00001
    )  # To ensure that the price never goes below zero!

SM2 = np.zeros((M + 1, Ite))
SM2[0] = S2
for t in range(1, M + 1):
    SM2[t] = SM2[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM2[t] = np.maximum(
        SM2[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [ ]:
deta_call = (merton_mc(SM1, 80, r, T, 0, type="C") - merton_mc(SM2, 80, r, T, 0, type="C"))/(2*epsilon)
delta_put = (merton_mc(SM1, 80, r, T, 0, type="P") - merton_mc(SM2, 80, r, T, 0, type="P"))/(2*epsilon)
print("EUR call delta:", delta_call.round(2))
print("EUR put delta:", delta_put.round(2))

NameError: ignored

For delta, the explanation is as same as the previous group work, call option with a positive delta and put option with a negative delta.

In [ ]:
S_f = 81

In [ ]:
Sf1 = S_f + epsilon
Sf2 = S_f - epsilon

In [ ]:
SMf1 = np.zeros((M + 1, Ite))
SMf1[0] = Sf1
for t in range(1, M + 1):
    SMf1[t] = SMf1[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SMf1[t] = np.maximum(
        SMf1[t], 0.00001
    )  # To ensure that the price never goes below zero!

SMf2 = np.zeros((M + 1, Ite))
SMf2[0] = Sf2
for t in range(1, M + 1):
    SMf2[t] = SMf2[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SMf2[t] = np.maximum(
        SMf2[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [ ]:
delta_call_sf = (merton_mc(SMf1, 80, r, T, 0, type="C") - merton_mc(SMf2, 80, r, T, 0, type="C"))/(2*epsilon)
delta_put_sf = (merton_mc(SMf1, 80, r, T, 0, type="P") - merton_mc(SMf2, 80, r, T, 0, type="P"))/(2*epsilon)
print("EUR call gamma:", ((delta_call_sf - delta_call)/(S_f-S0)).round(2))
print("EUR put gamma:", ((delta_put_sf - delta_put)/(S_f-S0)).round(2))

Option gamma is a measure of how much the option delta is expected to change for a one-point change in the price of the underlying asset. It is the second derivative of the option price with respect to the price of the underlying asset. Both call and put options have positive gamma.
Gamma decreases, approaching zero, as an option gets deeper in the money and delta approaches one. Gamma also approaches zero the deeper an option gets out of the money. Gamma is at its highest when the price is at the money.
In this case, EUR call has delta=0.52 and gamma=0.18, which mean if the underlying price increase by 1, delta will increase by 0.18 and become 0.70. The same for EUR put.


## **(11) Model Validator**



- Confirming whether the prices of the put and call from both the Heston Model and Merton Model does satisfy put-call parity.



In [ ]:
print('a. Put-Call parity for Heston Model method when rho is -0.30 is', 2.85 + np.round(80 * np.exp(-r * T),2) == 80 + 2.83)
print('b. Put-Call parity for Heston Model method when rho is -0.70 is',round(2.08,2) + np.round(80 * np.exp(-r * T),2) == 80 + round(3.46,2))
print('c. Put-Call parity for Merton method when jump intensity is 0.75 is ',round(8.91,2) + np.round(80 * np.exp(-r * T),2) == 80 + round(7.38,2))
print('d. Put-Call parity for Merton method when jump intensity is 0.25 is',round(7,2) + np.round(80 * np.exp(-r * T),2) == 80 + round(5.84,2))

**Put-Call Parity Checking for the Models and different Parameters**

| Model Method       | Parameter Name   | Parameter Value | Put-Call Parity |
|--------------------|------------------|-----------------|-----------------|
| Heston Model       | rho              | -0.30           | False           |
| Heston Model       | rho              | -0.70           | False           |
| Merton Model       | jump intensity   | 0.75            | False           |
| Merton Model       | jump intensity   | 0.25            | False           |

**Comment of Results:**

 Market frictions,Model assumptions or even parameter estimations that different from ideal conditions necessary to maintain a parity because the break of this possible reasons, the Put-Call parity was not maintained in both the Heston Model for rho values of -0.30 and -0.70, and the Merton method for jump intensities of 0.75 and 0.25.

### **Using the Heston Model for 7 different strikes prices**

In [ ]:
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 100  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)


call_heston=[]
arr = [0.85, 0.90, 0.95, 1, 1.05, 1.10,1.15]  # Define arr here
for a in arr:
  K=round(S0*a)
  c=heston_call_mc(S, K, r, T, sigma_v)
  call_heston.append(c)
  print("Call value for Heston Model strike",K," : ",round(c,2))

**Heston Model Call Values for 7 Different Strikes**

| Strike | Call Value  |
|--------|-------------|
|  85    | 15.34       |
|  90    | 10.77        |
|  95    | 6.75        |
| 100    | 3.63        |
| 105    | 1.65        |
| 110    | 0.65        |
| 115    | 0.23        |

**Comment**

As the strike prices increase the call values decrease and this shows the negative correlation between the strike price and the call option value.

### **Using the Merton Model for 7 different strikes**

In [ ]:
import numpy as np
from scipy.stats import norm

def merton_jump_call(S, K, T, r, sigma, lambd, delta):
    d1 = (np.log(S/K) + (r - lambd*delta + (sigma**2)/2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    C_BS = S * N_d1 - K * np.exp(-r*T) * N_d2
    C_jump = C_BS + lambd * delta * (np.exp(r*T) - 1)
    return C_jump


arr=[0.85,0.9,0.95,1,1.05,1.1,1.15]
lambd=0
delta=0
call_merton=[]
S = 100  # current stock price
T = 0.25  # time to expiry in years
r = 0.055  # risk-free interest rate
sigma = 0.35  # stock price volatility
mu = -0.5  # stock price drift rate
delta = 0.22  # jump size each
lambd = 0.25  # jump intensity parameter
for a in arr:
  K=round(S*a)
  c=merton_jump_call(S, K, T, r, sigma, lambd, delta)
  call_merton.append(c)
  print("Call value for Merton Model strike",K," : ",round(c,2))


**Merton Model Call Values for Seven Different Strikes**

| Strike | Call Value |
|--------|------------|
| 85     | 17.46      |
| 90     | 13.64      |
| 95     | 10.34      |
| 100    | 7.61       |
| 105    | 5.43       |
| 110    | 3.77       |
| 115    | 2.55       |

**Comment**

 The Heston and the Merton Model all showed that the option prices decrease as the moneyness level changes from Deep ITM to Deep OTM.

# **Step 2**

Question 13 a

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.3

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 1000000  # Number of simulations
dt = T / M  # Length of time step


# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)
# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)


def heston_call_american(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)
    value = payoff.copy()

    for t_step in range(M - 1, 0, -1):
        # Discounted expected value of continuing the option
        value = value * np.exp(-r * dt)
        continuation_value = np.maximum(0, S[t_step, :] - K)
        value = np.maximum(value, continuation_value)

        # Early exercise check
        payoff = np.maximum(0, S[t_step, :] - K)
        value = np.maximum(value, payoff)

    return np.mean(value)

def heston_put_american(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])
    value = payoff.copy()

    for t_step in range(M - 1, 0, -1):
        # Discounted expected value of continuing the option
        value = value * np.exp(-r * dt)
        continuation_value = np.maximum(0, K - S[t_step, :])
        value = np.maximum(value, continuation_value)

        # Early exercise check
        payoff = np.maximum(0, K - S[t_step, :])
        value = np.maximum(value, payoff)

    return np.mean(value)

print("American Call Price under Heston using a correlation value of -0.30: ", np.round(heston_call_american(S, 80, 0.055, 3/12, 0),2))
print("American Put Price under Heston using a correlation value of -0.30: ", np.round(heston_put_american(S, 80, 0.055, 3/12, 0),2))


Question 13 b

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

S0 = 80
r = 0.055
sigma = 0.35
T = 3/12
Ite = 500000  # Number of simulations (paths)
M = 1  # Number of steps
dt = T / M  # Time-step

lamb = 0.75  # Lambda of the model
mu = -0.6  # Mu
delta = 0.22  # Delta

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

def merton_american_mc(S, K, r, T, t, type="C"):
    if (type == "C"):
        payoff = np.maximum(0, S[-1, :] - K)
        value = payoff.copy()
    else:
        payoff = np.maximum(K - S[-1, :], 0)
        value = payoff.copy()

    for t_step in range(M - 1, 0, -1):
        value = value * np.exp(-r * dt)
        if (type == "C"):
            continuation_value = np.maximum(0, S[t_step, :] - K)
            value = np.maximum(value, continuation_value)
            payoff = np.maximum(0, S[t_step, :] - K)
        else:
            continuation_value = np.maximum(0, K - S[t_step, :])
            value = np.maximum(value, continuation_value)
            payoff = np.maximum(K - S[t_step, :], 0)

        value = np.maximum(value, payoff)

    return np.mean(value)

print("American Call Price under Merton: ", np.round(merton_american_mc(SM, 80, r, T, 0, type="C"),2))
print("American Put Price under Merton: ", np.round(merton_american_mc(SM, 80, r, T, 0, type="P"),2))


We repeated the pricing for **American options** using the Heston Model and Merton Model. The key differences observed from the European option pricing:

*   American options allow for early exercise, affecting their pricing compared to European options.

*   Due to the potential for early exercise, American call options are more expensive than their European counterparts due to added flexibility.





Question 15

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [ ]:
np.random.seed(42)

In [ ]:
import numpy as np

S0 = 80
r = 0.055
sigma = 0.35
T = 3/12
Ite = 10000000  # Number of simulations (paths)
M = 1  # Number of steps
dt = T / M  # Time-step

lamb = 0.75  # Lambda of the model
mu = -0.6  # Mu
delta = 0.25  # Delta

barrier = 65  # Barrier level for down-and-in option

SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

def merton_dai_option(S, K, r, T, t, barrier):
    active_paths = np.any(S <= barrier, axis=0)
    payoff = np.maximum(K - S[-1, :], 0)
    payoff[~active_paths] = 0  # Set payoff to zero for paths that did not hit the barrier
    average = np.mean(payoff)
    return np.exp(-r * (T - t)) * average

# Example usage:
K = 65  # Strike price
t = 0  # Option start time
dai_price = merton_dai_option(SM, 65, r, T, t, 65)
print("Down-and-In put Option Price:", dai_price.round(2))


In [ ]:
print("European Put Price under Merton: ", merton_mc(SM, 65, r, T, 0, type="P").round(2))

In this particular case, both have nearly the same price. In theory, DAI put option price is lower than the normal put option. Since we use the one step in this case, this makes the DAI put option nearly the same as the normal put option when we only compare the price with the barrier (also strike) at the expiration. If using more steps in stock price simulation, the result is different.